## 简介

PyTorch Geometric Library (简称 PyG) 是一个基于 PyTorch 的图神经网络库，地址是：https://github.com/rusty1s/pytorch_geometric。

安装网址在: https://pytorch-geometric.readthedocs.io/en/latest/notes/installation.html

相比于简单的文本和图像，这种网络类型的非结构化的数据非常复杂，处理它的难点包括：

1. 图的大小是任意的，图的拓扑结构复杂，没有像图像一样的空间局部性。

2. 图没有固定的节点顺序，或者说没有一个参考节点。

3. 图经常是动态图，而且包含多模态的特征。


比较好的教程：

- [PytorchGeometricTutorial](https://github.com/AntonioLonga/PytorchGeometricTutorial)

## torch_geometric.data.Data

在`PyG`中，如果想要构建图，我们需要两个要素：节点和边。`PyG`提供了`torch_geometric.data.Data()`用来构建图，包括`5`个属性，每一个属性都不是必须的，可以为空。

1. `x`：用于存储每个节点的特征，形状是`[num_nodes, num_node_features]`，行表示节点个数，列表示每个节点对应的特征个数。

2. `edge_index`: 用于存储节点之间的边，形状是`[2, num_edges]`，表示的是每条边对应的两个节点。节点连接信息要以`COO`格式进行存储。在`COO`格式中，`COO list`是一个`2*E`维的`list`。第一个维度的节点是源节点(`source nodes`)，第二个维度中是目标节点(`target nodes`)，连接方式是由源节点指向目标节点。对于无向图来说，存贮的`source nodes`和`target node`是成对存在的。

3. `pos`: 存储节点的坐标，形状是`[num_nodes, num_dimensions]`。

4. `y`: 存储样本标签。如果是每个节点都有标签，那么形状是`[num_nodes, *]`；如果是整张图只有一个标签，那么形状是`[1, *]`。

5. `edge_attr`: 存储边的特征。形状是`[num_edges, num_edge_features]`。

实际上，Data对象不仅仅限制于这些属性，我们可以通过data.face来扩展Data，以张量保存三维网格中三角形的连接性。

下面这个例子是未加权无向图(未加权指的是边上没有权值)，包括3个节点和4条边。

<img src="../../images/12-graph_example1.png" width="40%">

由于是无向图，因此有4条边：(0 -> 1), (1 -> 0), (1 -> 2), (2 -> 1)。每个节点都有自己的特征，上面这个图可以使用`torch_geometric.data.Data`来表示如下：

In [1]:
import torch
from torch_geometric.data import Data
# 由于是无向图，因此有 4 条边：(0 -> 1), (1 -> 0), (1 -> 2), (2 -> 1)
edge_index = torch.tensor([[0, 1, 1, 2],
                           [1, 0, 2, 1]], dtype=torch.long)
# 节点的特征                           
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

data = Data(x=x, edge_index=edge_index)
data

Data(edge_index=[2, 4], x=[3, 1])

对于edge_index的表示，也可以换一种写法：

In [2]:
import torch
from torch_geometric.data import Data

edge_index = torch.tensor([[0, 1],
                           [1, 0],
                           [1, 2],
                           [2, 1]], dtype=torch.long)
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

data = Data(x=x, edge_index=edge_index.t().contiguous())
data

Data(edge_index=[2, 4], x=[3, 1])

但是需要先将edge_index转置，然后调用contiguous()方法来使得其在内存空间中是连续的，更多的可以参考https://zhuanlan.zhihu.com/p/64551412。

最后再复习一遍，Data中最基本的 4 个属性是x、edge_index、pos、y，我们一般都需要这 4 个参数。

除此之外，Data还提供了一些有用的函数:

In [3]:
print(data.keys)
print(data['x'])
for key, item in data:
    print("{} is in data".format(key))
print(data.num_nodes) # 查看节点个数
print(data.num_edges) # 查看边的个数
print(data.num_node_features) # 查看节点的特征数
# print(data.has_isolated_nodes())  # 是否有孤立节点
# print(data.has_self_loops())  # 是否有环
# print(data.is_directed())  # 

# Transfer data object to GPU.
# device = torch.device('cuda')
# data = data.to(device)

['x', 'edge_index']
tensor([[-1.],
        [ 0.],
        [ 1.]])
edge_index is in data
x is in data
3
4
1


## Dataset 与 DataLoader

`PyG`的`Dataset`继承自`torch.utils.data.Dataset`，自带了很多图数据集，我们以`TUDataset`为例，通过以下代码就可以加载数据集，`root`参数设置数据下载的位置。

In [4]:
from torch_geometric.datasets import TUDataset
dataset = TUDataset(root='/tmp/ENZYMES', name='ENZYMES')
print(len(dataset))  # 图的数量。
print(dataset.num_classes)  # 类别个数
print(dataset.num_node_features)  # 节点特征数

600
6
3


In [5]:
data = dataset[0]
print(data)
print(data.is_undirected())

Data(edge_index=[2, 168], x=[37, 3], y=[1])
True


可以看到，第一张图中包含了37个节点，每个节点有3个特征，总共有168/2=84条无向边，整个的这张图为一个类别。

我们可以用切片的方式对数据集进行切分


In [6]:
dataset = dataset.shuffle() # 打乱数据。

train_dataset = dataset[:540]
test_dataset = dataset[540:]
print(len(train_dataset))
print(len(test_dataset))

540
60


## Mini-Batches

`PyG`通过创建稀疏的对角邻接矩阵，并在节点维度中连接**特征矩阵**和**label**矩阵，实现了在`mini-batch`的并行化。

把一个batch的所有图都拼接在一起形成一个大图。

$$
\mathbf{A}=\left[\begin{array}{ccc}
\mathbf{A}_{1} & & \\
& \ddots & \\
& & \mathbf{A}_{n}
\end{array}\right], \quad \mathbf{X}=\left[\begin{array}{c}
\mathbf{X}_{1} \\
\vdots \\
\mathbf{X}_{n}
\end{array}\right], \quad \mathbf{Y}=\left[\begin{array}{c}
\mathbf{Y}_{1} \\
\vdots \\
\mathbf{Y}_{n}
\end{array}\right]
$$

当然在这个大图里面，来自不同样本的节点之间是不存在连接的。torch_geometric.data.DataLoader 类帮我们实现好了batch里面样本的拼接过程，我们直接拿来用就可以了。

In [7]:
from torch_geometric.datasets import TUDataset
from torch_geometric.data import DataLoader

dataset = TUDataset(root='/tmp/ENZYMES', name='ENZYMES', use_node_attr=True)
loader = DataLoader(dataset, batch_size=32, shuffle=True)
Is_Print = True
for batch in loader:
    batch
    if Is_Print:
        print(batch)
        # >>> Batch(batch=[1082], edge_index=[2, 4066], x=[1082, 21], y=[32])
        print(batch.batch)
        print(len(batch.batch))
        Is_Print = False
    batch.num_graphs
    # >>> 32

Batch(batch=[1002], edge_index=[2, 3948], ptr=[33], x=[1002, 21], y=[32])
tensor([ 0,  0,  0,  ..., 31, 31, 31])
1002


此外每个`batch`还有一个特殊的列向量：`batch.batch`，它表示在这个批次的大图里面，各个节点在这个批次的第几个子图里面。

In [8]:
from torch_scatter import scatter_mean
from torch_geometric.datasets import TUDataset
from torch_geometric.data import DataLoader

dataset = TUDataset(root='/tmp/ENZYMES', name='ENZYMES', use_node_attr=True)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

for data in loader:
    data
    # >>> Batch(batch=[1082], edge_index=[2, 4066], x=[1082, 21], y=[32])

    data.num_graphs
    # >>> 32
    # scatter函数比较复杂，细节之后解释；总之效果为按照顶点属于哪张图，将属于同张图中的顶点计算所有顶点x的特征平均值，存储在变量x中
    x = scatter_mean(data.x, data.batch, dim=0)
    print(x.size())
    
    # >>> torch.Size([32, 21])

torch.Size([32, 21])
torch.Size([32, 21])
torch.Size([32, 21])
torch.Size([32, 21])
torch.Size([32, 21])
torch.Size([32, 21])
torch.Size([32, 21])
torch.Size([32, 21])
torch.Size([32, 21])
torch.Size([32, 21])
torch.Size([32, 21])
torch.Size([32, 21])
torch.Size([32, 21])
torch.Size([32, 21])
torch.Size([32, 21])
torch.Size([32, 21])
torch.Size([32, 21])
torch.Size([32, 21])
torch.Size([24, 21])


## 实战一个图神经网络

In [9]:
for i in range(100): # 循环下载
    try:
        from torch_geometric.datasets import Planetoid
        dataset = Planetoid(root='.\data\Cora', name='Cora')
    except:
        print('pass')
# from torch_geometric.datasets import Planetoid
# dataset = Planetoid(root='/tmp/Cora', name='Cora')
print(dataset)

Cora()


In [10]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

In [11]:
class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)
        
    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
print(data)
model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

Data(edge_index=[2, 10556], test_mask=[2708], train_mask=[2708], val_mask=[2708], x=[2708, 1433], y=[2708])


`train_mask`——指明训练集中的节点（可以看到，在这个数据集中，训练集里有`140`个节点）

`val_mask`——指明验证集中的节点（可以看到，在这个数据集中，验证集里有`500`个节点）

`test_mask`——指明测试集中的节点（可以看到，在这个数据集中，测试集里有`1000`个节点）

测试

In [13]:
model.eval()
pred = model(data).argmax(dim=1)
correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
acc = int(correct) / int(data.test_mask.sum())
print('Accuracy: {:.4f}'.format(acc))

Accuracy: 0.8010
